# Random Forest

In [84]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
import sys; sys.path.append('/Users/uqhkamel/PhD/Code/AKI_mimiciv/mimic-code-main/mimic-iv/src')
import pandas as pd
import matplotlib.pyplot as plt


import numpy as np
import sqlite3

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score


from pickle import dump
from dfwiz import dfwiz
from dfwiz import dfwiz_compare

from sklearn.ensemble import RandomForestClassifier

from skopt import BayesSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import StratifiedKFold


from sklearn.metrics import recall_score


# from sklearn.pipeline import Pipeline


from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE



from sklearn.experimental import enable_iterative_imputer

from sklearn.impute import IterativeImputer

from sklearn.preprocessing import MinMaxScaler

from sklearn.utils import resample

import copy

from sklearn import metrics


from utils.vis import spy, look, plot_nunique, plot_dists
from utils.processing import sort, impute, replace_inf, drop_empty, select, drop_by_nunique, scale, melt, unmelt, \
                             remove_outliers, get_categories, filter_categorical, onehot, filter_regex, match, cap,get_dates
from utils.pipelines import scale_impute_via_pipeline

In [86]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import numpy as np
pd.set_option("display.max_columns", None)

In [87]:
# global variables representing experiment parameters
EXPERIMENT = 'Processing Demo'
IMPUTE_NUM = 'constant'
IMPUTE_CAT = 'other'
FIGSIZE    = [12,3]

# parameter dict
params = {
    'experiment':EXPERIMENT,
    'figsize'   :FIGSIZE,
    'impute_num':IMPUTE_NUM,
    'impute_cat':IMPUTE_CAT,
}

In [88]:

import scipy as sp

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Remove warnings
import warnings
warnings.filterwarnings('ignore')

# Plot settings
plt.style.use('seaborn')
sns.set_theme(style="ticks")
mpl.rcParams['figure.figsize'] = (10,6)

# Title
mpl.rcParams['figure.titlesize'] = 22
mpl.rcParams['figure.titleweight'] = 'bold'
mpl.rcParams['axes.titlesize'] = 22
mpl.rcParams['axes.titleweight'] = 'bold'
mpl.rcParams['axes.titlepad'] = 20

# Axes labels
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['axes.labelweight'] = 'bold'

# Grid and thicks
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.left'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.grid'] = True
mpl.rcParams['axes.grid.axis'] = 'y'
#mpl.rcParams['axes.xmargin'] = 0
mpl.rcParams['ytick.left'] = False

# Legend
mpl.rcParams['legend.facecolor'] = 'w'
mpl.rcParams['legend.title_fontsize'] = 14
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['legend.frameon'] = True
mpl.rcParams['legend.framealpha'] = 1
mpl.rcParams['legend.fancybox'] = True
mpl.rcParams['legend.facecolor'] = 'white'
mpl.rcParams['legend.edgecolor'] = 'blue'
mpl.rcParams['legend.borderpad'] = 0.6

# Other
mpl.rcParams['lines.linewidth'] = 2.5
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['scatter.edgecolors'] = None




_________
### upsampler func def

In [89]:
from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline

class upsampler(BaseEstimator, TransformerMixin): 
    def __init__(self):
        return None
    
    def fit(self, X, y = None):
        return self
    def transform(self, X, y = None):
        return X

    def sample(self, X, y = None):
        X = np.array(X)
        y = np.array(y)
        if len(y[y == 0]) < len(y[y == 1]):
            X1, y1 = resample(X[y[y == 0]], y[y == 0], random_state=0, n_samples=len(y[y == 1]))
            X2, y2 = X[y[y == 1]], y[y == 1]
        else:
            print(X[y[y == 0]].shape)
            X1, y1 = resample(X[y[y == 1]], y[y == 1], random_state=0, n_samples=len(y[y == 0]))
            X2, y2 = X[y[y == 0]], y[y == 0]
        X_out = np.vstack((X1, X2))
        y_out = np.hstack((y1, y2))  

        return X_out, y_out


_________
### accuracy func def

In [90]:
def confusion_matrix_plot(y, y_pred, y_pred_proba):

    fpr, tpr, _ = metrics.roc_curve(y,   y_pred_proba[::,1])
    score = metrics.roc_auc_score(y,  y_pred_proba[::,1])

    #create ROC curve
    plt.plot(fpr,tpr,label="AUC="+str(round(score,2)))
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc=4)
    plt.show()


    cm = confusion_matrix(y, y_pred)
    plt.figure(figsize=(7,7))
    plt.clf()
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Wistia)
    classNames = ['Negative','Positive']
    plt.title('Confusion Matrix')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    tick_marks = np.arange(len(classNames))
    plt.xticks(tick_marks, classNames, rotation=45)
    plt.yticks(tick_marks, classNames)
    s = [['TN','FP'], ['FN', 'TP']]
    
    for i in range(2):
        for j in range(2):
            plt.text(j,i, str(s[i][j])+" = "+str(cm[i][j]))
    plt.show()
    
    accuracy = accuracy_score(y, y_pred)

    # print("Accuracy: %.2f%%" % (accuracy * 100.0))


    cr = classification_report(y, y_pred)
    print("\r\n"+"Classification report"+"\r\n")
    print(cr)

    print("\r\n_________________________________________")
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    specificity = tn / (tn+fp)
    print("\r\n"+"Specificity"+"\r\n")
    print(round(specificity,2))

    print("\r\n_________________________________________")



In [91]:
from sklearn.utils import resample

def up_sample(X_train_raw, y_train_raw,col_name):

    # upsampling X_train and y_train
    df_upsampled = pd.merge(X_train_raw, y_train_raw, left_index=True, right_index=True)

    X_minority = df_upsampled[df_upsampled[col_name]==1]
    X_majority = df_upsampled[df_upsampled[col_name]!=1]

    n_samples = X_majority.shape[0]
    X_minority_upsampled = resample(X_minority,
                                    replace=True,     # sample with replacement
                                    n_samples=n_samples,    # to match majority class
                                    random_state=42) # reproducible results

    df_upsampled = pd.concat([X_majority, X_minority_upsampled]).sample(frac=1)

    y_train_out = df_upsampled[[col_name]]
    X_train_out = df_upsampled.drop([col_name], axis=1)

    return X_train_out, y_train_out

_________
### define cross validation function

In [92]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.model_selection import StratifiedKFold



def param_graph(X_train, y_train, pipe, param_grid, cv=5, max_iter = 5, sample_ratio = 0.2, refit=True, use_error=True, multi_class=False, average_metric='macro'):

    print("This search selects lower indexes of search list if their score is within the error of maximum score.")
    print("Putting parameters for less complicated model on the left side of the grid lists leads to better generalisation. ")
    print(" ")

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    n_train = int(sample_ratio * len(y_train))
    X_train_s, y_train_s  = resample(X_train, y_train, n_samples=n_train, stratify=y_train)

    best_score = {}
    best_params = {}
    for k, v in param_grid.items():
        # best_params[k] = v[int(len(v)/2)-1]
        best_params[k] = v[0]
    best_params_m1 = best_params.copy()
    print("start_params:", best_params)

    score = {}
    score_std = {}

    for i_iter in range(max_iter):
        print("_"*100)
        print("Iteration", i_iter)

        for k, v in param_grid.items():

            best_params1 = best_params.copy()
            del best_params1[k]  

            score[k] = v.copy()
            score_std[k] = v.copy()

            for i_param, val_param in enumerate(v):
                cv_sc = np.zeros(cv)

                for i_cv in range(cv):

                    X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train_s, y_train_s, test_size=0.2, stratify=y_train_s, shuffle=True) # 80% training and 20% test

                    p1 = copy.deepcopy(pipe)
                    p1.set_params(**best_params1)
                    params2 = {k:val_param}
                    p1.set_params(**params2)

                    p1.fit(X_train2, y_train2.ravel())
                    # X,y = p1.named_steps['resample'].fit_resample(X_test2, y_test2)
                    X,y = X_test2, y_test2
                    # y_pred_proba = p1.predict_proba(X)
                    # cv_sc[i_cv] = metrics.roc_auc_score(y,  y_pred_proba[::,1])
                    y_pred = p1.predict(X)
                    if(multi_class):
                        cv_sc[i_cv] = metrics.f1_score(y, y_pred, average=average_metric)
                    else:
                        cv_sc[i_cv] = metrics.f1_score(y, y_pred)

                    i_cv = i_cv + 1

                score[k][i_param] = cv_sc.mean()
                score_std[k][i_param] = cv_sc.std()

            print("")
            print(k)
            print(v)
            print(score[k])

            best_params[k] = v[np.argmax(score[k])]
            best_score[k] = score[k][np.argmax(score[k])]

            if use_error:
                for i_b in  range(np.argmax(score[k]),-1,-1):
                    err1 = (score_std[k][i_b] + score_std[k][v.index(best_params[k])] ) / 4
                    # print("err1")
                    max_del = max(score[k]) - err1
                    # print( i_b, score[k][i_b], max(score[k]), err1, max_del )
                    if score[k][i_b] >= max_del:
                        best_params[k] = v[i_b]
                        best_score[k] = score[k][i_b]

            print("best_param:",  v[np.argmax(score[k])], "score:", max(score[k]))
            print("selected_param:",  best_params[k], "score:", best_score[k])
            

        
        print("")
        print("best_params =", best_params)
        print("")
        if best_params_m1 == best_params:
            print("")
            print("")
            print("Early stop. No improvement in the last iteration.")
            break
        best_params_m1 = best_params.copy()

    param_graph_plot(score)

    if refit:
        print("Refitting final model...")
        pipeline_final = copy.deepcopy(pipe)
        pipeline_final.set_params(**best_params)
        pipeline_final.fit(X_train, y_train.values.ravel())
    else:
        pipeline_final = None

    return score, best_params, pipeline_final
    

def param_graph_plot(score):
    ax = {}
    fig = {}
    for i, (k, v) in enumerate(score.items()):
        fig[k], ax[k] = plt.subplots()

    for k, v in score.items():
        x = score[k]
        y = v
        ax[k].plot(x,y,"-o", label="Score")
        # ax[k].set_ylim([0.5, 1])
        ax[k].set_title(k)
        ax[k].legend()

________
### Define upsampler

In [93]:

from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from sklearn.utils import resample


class upsampler(BaseEstimator): 
    def __init__(self):
        return None

    def fit_resample(self, X, y = None):
        X = np.array(X)
        y = np.array(y).ravel()
        if len(y[y == 0]) < len(y[y == 1]):
            X1, y1 = resample(X[y == 0], y[y == 0], random_state=0, n_samples=len(y[y == 1]))
            X2, y2 = X[y == 1], y[y == 1]
        else:
            X1, y1 = resample(X[y == 1], y[y == 1], random_state=0, n_samples=len(y[y == 0]))
            X2, y2 = X[y == 0], y[y == 0]
        X_out = np.vstack((X1, X2))
        y_out = np.hstack((y1, y2))  
        return X_out, y_out

In [94]:
# create a database connection
sqluser = 'uqhkamel'
dbname = 'mimiciv'
schema_name = 'mimic_derived'

In [95]:
# Connect to local postgres version of mimic
con = psycopg2.connect(dbname=dbname, user=sqluser)
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema_name))

In [96]:
query = "select * from all_scr_preadmission_75_JOIN_180_0days"
data = pd.read_sql_query(query,con,index_col=['stay_id','subject_id'])
data.drop('hadm_id', inplace=True, axis=1)

## Rename columns

In [97]:
data.rename(columns={
'egfr_epi_scr_max':'eGFR',
'pt_max':'Prothrombin time',
'invasive_vent':'invasive ventilation',
'congestive_heart_failure':'congestive heart failure',
'diabetes_type2':'diabetes Type2',
'resp_rate_max':'respiratory rate max',
'sbp_max':'systolic blood pressure max',
'heart_rate_max':'heart rate max',
'dbp_max':'diastolic blood pressure max',
'spo2_min':'oxygen saturation min',
'spo2_max':'oxygen saturation max',
'bicarbonate_max':'bicarbonate max',
'bicarbonate_min':'bicarbonate min',
'ckd':'chronic kidney disease',
'wbc_max':'white blood count max',
'wbc_min':'white blood count min',
'bun_min':'blood urine nitrogen min',
'creatinine_min':'creatinine min',
'creatinine_max':'creatinine max',
'hematocrit_min':'hematocrit min',
'sodium_max':'sodium max',
'chloride_max':'chloride max',
'hemoglobin_min':'hemoglobin min',
'aniongap_max':'aniongap max',
'supplemental_oxygen':'supplemental Oxygen',
'calcium_max':'calcium max',
'urineoutput_24hr':'urine output',
'myocardial_infarct':'myocardial infarction',
'aniongap_min':'aniongap min',
'calcium_min':'calcium min'
}, inplace=True)

## Remove useless columns

In [98]:
drop_X   = [
'potassium_min',
'temperature_min',
'glucose_max',
'glucose_min',
'sodium_min',
'temperature_mean',
'egfr_mdrd_scr_max',
'pt_min',
'inr_min',
'inr_max',
'weight_admit',
'weight_min',
'ptt_max',
'ptt_min',
'platelets_max',
'resp_rate_min',
'resp_rate_mean',
'sbp_mean',
'sbp_min',
'chloride_min',
'heart_rate_mean',
'heart_rate_min',
'dbp_min',
'dbp_mean',
'non_invasive_vent',
'wbc_bd_min',
'wbc_bd_max',
'chronic_kidney_disease',
'arbs_acei',
'hematocrit_max',
'bun_max'
]


data.drop(drop_X, inplace=True, axis=1)

In [99]:
data['ethnicity'] = data['ethnicity'].replace(['OTHER'],np.nan)
data['ethnicity'] = data['ethnicity'].replace(['UNKNOWN'],np.nan)
data['ethnicity'] = data['ethnicity'].replace(['UNABLE TO OBTAIN'],np.nan)
data['ethnicity'] = data['ethnicity'].replace(['UNABLE TO OBTAIN'],np.nan)
data['ethnicity'] = data['ethnicity'].replace(['AMERICAN INDIAN/ALASKA NATIVE'],np.nan)

data = data.fillna(value=np.nan)


In [100]:
data.shape

(46300, 79)

In [101]:
data.shape

(46300, 79)

In [102]:
data[data['min_day_rrt_present']<=1].shape

(1376, 79)

In [103]:
outcome_var = []
outcome_var.append('min_day_rrt_present')


first_24h = 1
data= data[data[outcome_var].min(axis=1)>first_24h]

In [104]:
# data[data['ckd']==1].shape

In [105]:
# data = data[data['ckd']==0]

In [106]:
data[data['kidney_transplant']==1].shape

(514, 79)

In [107]:
data = data[data['kidney_transplant']==0]

In [108]:
# data = data[data['egfr_mdrd_scr']>60]

In [109]:
# data[data.egfr_mdrd_scr<60].shape

In [110]:
data.head(5)

,,aki_kdigo_grade_1,aki_kdigo_grade_2,aki_kdigo_grade_3,day_detection_kdigo_grade_1,day_detection_kdigo_grade_2,day_detection_kdigo_grade_3,aki_mkdigo_grade_1,aki_mkdigo_grade_2,aki_mkdigo_grade_3,day_detection_mkdigo_grade_1,day_detection_mkdigo_grade_2,day_detection_mkdigo_grade_3,age,female,ethnicity,chronic kidney disease,is_mdrd,egfr_epi_scr,egfr_mdrd_scr,kidney_transplant,congestive heart failure,diabetes Type2,hypertension,obesity_icd,peripheral_vascular_disease,chronic_liver_disease,mild_liver_disease,severe_liver_disease,myocardial infarction,chronic_pulmonary_disease,chronic_heart_failure,sepsis,hematocrit min,hemoglobin min,hemoglobin_max,platelets_min,white blood count min,white blood count max,albumin_min,albumin_max,globulin_min,globulin_max,total_protein_min,total_protein_max,aniongap min,aniongap max,bicarbonate min,bicarbonate max,blood urine nitrogen min,calcium min,calcium max,chloride max,creatinine min,creatinine max,sodium max,potassium_max,Prothrombin time,thrombin_min,thrombin_max,bilirubin_total_min,bilirubin_total_max,eGFR,heart rate max,systolic blood pressure max,diastolic blood pressure max,respiratory rate max,temperature_max,oxygen saturation min,oxygen saturation max,cyclosporine,bmi,urine output,supplemental Oxygen,invasive ventilation,hfnc,tracheostomy,min_day_rrt_present,min_day_rrt_active,weight_max
stay_id,subject_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
30011241,14731650,0,0,0,9999999.0,9999999.0,9999999.0,0,0,0,9999999.0,9999999.0,9999999.0,82.0,1,NaN,0,1,52.0,53.0,0,0,0,1,0,0,0,0,0,1,1,0,0,25.7,8.0,8.4,144.0,11.7,17.0,2.4,2.4,NaN,NaN,NaN,NaN,10.0,11.0,25.0,27.0,17.0,7.8,7.8,106.0,0.8,1.0,139.0,3.5,12.8,NaN,NaN,0.5,0.6,52.0,91.0,147.0,63.0,22.0,37.06,94.0,100.0,0,NaN,90.0,0,1,0,0,99999999.0,99999999.0,85.0
30016265,16142166,0,0,0,9999999.0,9999999.0,9999999.0,0,0,0,9999999.0,9999999.0,9999999.0,79.0,1,NaN,0,0,90.0,109.0,0,1,1,1,0,1,0,0,0,0,0,0,0,21.0,8.0,11.8,80.0,10.2,19.5,NaN,NaN,NaN,NaN,NaN,NaN,7.0,11.0,24.0,24.0,13.0,7.7,7.7,115.0,0.4,0.5,142.0,4.5,16.7,NaN,NaN,NaN,NaN,92.0,85.0,152.0,74.0,26.0,NaN,95.0,100.0,0,47.94,150.0,1,1,0,0,99999999.0,99999999.0,94.1
30020989,17594087,1,0,0,1.0,9999999.0,9999999.0,1,0,0,1.0,9999999.0,9999999.0,85.0,0,WHITE,1,1,36.0,38.0,0,0,0,1,0,0,0,0,0,0,1,0,0,24.6,8.6,10.4,486.0,9.4,11.8,3.3,3.3,NaN,NaN,NaN,NaN,13.0,16.0,22.0,24.0,37.0,7.6,8.2,108.0,1.3,1.7,141.0,4.5,16.6,NaN,NaN,0.3,0.3,36.0,107.0,182.0,86.0,25.0,38.33,96.0,100.0,0,NaN,100.0,0,0,0,0,99999999.0,99999999.0,74.1
30022756,19658144,0,0,0,9999999.0,9999999.0,9999999.0,1,1,0,1.0,9999999.0,9999999.0,89.0,1,WHITE,0,1,68.0,71.0,0,0,0,1,0,0,0,0,0,0,0,0,0,40.2,12.4,14.1,212.0,8.3,9.1,4.2,4.2,NaN,NaN,NaN,NaN,18.0,18.0,24.0,25.0,20.0,10.0,10.0,107.0,0.4,0.4,145.0,3.9,11.4,NaN,NaN,0.9,0.9,92.0,82.0,155.0,100.0,22.0,37.94,94.0,97.0,0,NaN,110.0,0,0,0,0,99999999.0,99999999.0,62.0
30023204,18738693,1,0,0,1.0,9999999.0,9999999.0,1,1,0,1.0,9999999.0,9999999.0,46.0,0,WHITE,0,1,51.0,47.0,0,0,0,1,0,0,0,0,0,0,0,0,1,19.7,6.6,8.3,440.0,15.2,27.0,2.8,2.8,NaN,NaN,NaN,NaN,13.0,17.0,19.0,22.0,23.0,7.4,8.5,104.0,1.4,1.6,132.0,5.5,10.9,NaN,NaN,0.7,0.7,51.0,125.0,128.0,77.0,23.0,37.28,91.0,100.0,0,NaN,50.0,1,0,0,0,99999999.0,99999999.0,86.1


In [111]:
# data.dropna(axis=1, thresh = int(0.3*data.shape[0]), inplace=True)

In [112]:
# pd.set_option('display.max_rows', None)

In [113]:
# data.isna().sum()/len(data)*100

In [114]:
# prediction_window = 3

# data.loc[(((data['aki_kdigo_grade_1']== 1)| (data['aki_kdigo_grade_2']== 1) | (data['aki_kdigo_grade_3']==1)) \
#     &( (data['day_detection_kdigo_grade_1']<=prediction_window)| (data['day_detection_kdigo_grade_2']<=prediction_window) | (data['day_detection_kdigo_grade_3']<=prediction_window)) \
#         |(data['min_day_rrt_present']<= prediction_window)), 'outcome'] = 1


# data.loc[data.outcome.isna(),'outcome']=0

In [115]:
prediction_window = 3

data.loc[(( (data['aki_kdigo_grade_1']== 1)) \
    &( (data['day_detection_kdigo_grade_1']<=prediction_window))), 'outcome'] = 1


data.loc[data.outcome.isna(),'outcome']=0

In [116]:
# drop_X   = [
# 'day_detection_kdigo_grade_1',
# 'day_detection_kdigo_grade_2',
# 'day_detection_kdigo_grade_3',
# 'day_detection_mkdigo_grade_1',
# 'day_detection_mkdigo_grade_2',
# 'day_detection_mkdigo_grade_3',
# 'min_day_rrt_active',
# 'min_day_rrt_present'
# # 'ckd',
# # 'chronic_kidney_disease'
# ]
# # CRP and vomit_nausea as they had mostly empty

# data.drop(drop_X, inplace=True, axis=1)

## Missingness percentage

In [117]:
perc = 20 # remove rows with NaN is 80 or more in each row
min_count =  int(((100-perc)/100)*data.shape[0])
data.dropna(axis=1, thresh = min_count, inplace=True)

In [118]:
# data.reset_index().drop_duplicates(subset=['stay_id','subject_id','hadm_id']).set_index(['stay_id','subject_id','hadm_id'])

In [119]:
# # remove unpopulated columns
# data.pipe(sort)\
#               .pipe(replace_inf).pipe(drop_empty)

In [120]:
# split by column type
data_num = data.pipe(sort).pipe(replace_inf).pipe(drop_empty).pipe(select, 'numerical')

data_cat = data.pipe(sort).pipe(replace_inf).pipe(drop_empty).pipe(select, 'categorical')

In [121]:
# data_cat = data_cat.pipe(filter_categorical, cutoff=20, plot=False)\
#                                             .pipe(sort).pipe(spy, title='Before onehot', figsize=[12,4])\
#                                             .fillna('other').pipe(onehot)
# data_cat = data_cat.fillna('other').pipe(onehot)
data_cat = pd.get_dummies(data_cat,prefix=[''], prefix_sep='', columns = ['ethnicity'], drop_first=True)

In [122]:
# processed = pd.merge(data_num, data_cat, left_index=True, right_index=True)
processed = pd.merge(data_num, data_cat, left_index=True, right_index=True, how='inner')

In [123]:
processed['is_mdrd'].sum()

16984

In [124]:
tmp1 = processed[processed['is_mdrd']==0]
tmp1.aki_kdigo_grade_1.value_counts()

0    16596
1    10830
Name: aki_kdigo_grade_1, dtype: int64

In [125]:
# processed.is_mdrd.value_counts()
processed['is_mdrd'].sum()/len(processed)*100

38.243638820085565

In [126]:
processed2 = processed.copy()
processed.drop(['egfr_epi_scr','egfr_mdrd_scr'], inplace=True, axis=1)


In [127]:
(processed[processed['aki_kdigo_grade_1']==1].shape[0])/processed.shape[0]

0.41200180139608195

In [128]:
processed[data['day_detection_kdigo_grade_1']<=1].shape

(12824, 69)

In [129]:
outcome_var = ['day_detection_kdigo_grade_1']

first_24h = 1
processed= processed[processed[outcome_var].min(axis=1)>first_24h]

In [130]:
drop_X   = [
'day_detection_kdigo_grade_1',
'day_detection_kdigo_grade_2',
'day_detection_kdigo_grade_3',
'day_detection_mkdigo_grade_1',
'day_detection_mkdigo_grade_2',
'day_detection_mkdigo_grade_3',
'min_day_rrt_active',
'min_day_rrt_present'
# 'ckd',
# 'chronic_kidney_disease'
]
# CRP and vomit_nausea as they had mostly empty

processed.drop(drop_X, inplace=True, axis=1)

In [131]:
drop_X   = [
    'aki_kdigo_grade_1',
    'aki_mkdigo_grade_1',

    'aki_kdigo_grade_2',
    'aki_mkdigo_grade_2',

    'aki_kdigo_grade_3',
    'aki_mkdigo_grade_3',
    'is_mdrd'

]
 
select_y = ['outcome']

processed_X = processed.pipe(filter_regex, drop_X+select_y)
processed_Y = processed.filter(regex='|'.join(select_y))
raw_Y = data_num.pipe(replace_inf).pipe(drop_empty).filter(regex='|'.join(select_y)).pipe(remove_outliers)
df_y = raw_Y[select_y]


df_X, df_y = match(processed_X, df_y)

In [132]:
df_y.outcome.value_counts()

0.0    26113
1.0     5473
Name: outcome, dtype: int64

In [133]:
10404/df_y.shape[0]

0.3293864370290635

In [134]:
processed.shape

(31586, 61)

In [135]:
categorical_X   = [
'chronic_heart_failure',
'chronic kidney disease',
'diabetes Type2',
'kidney_transplant',
'obesity_icd',
'chronic_heart_failure',
'myocardial infarction',
'chronic_pulmonary_disease',
'peripheral_vascular_disease',
'severe_liver_disease',
'mild_liver_disease',
'congestive heart failure',
'chronic_liver_disease',
'female',
'supplemental Oxygen',
'BLACK/AFRICAN AMERICAN',
'HISPANIC/LATINO',
'invasive ventilation',
'WHITE',
'sepsis',
'hypertension',
'hfnc',
'cyclosporine',
'tracheostomy'
]

# numerical_data_X = df_X.drop(categorical_X, axis=1)
# numerical_data_X = numerical_data_X.reindex(sorted(numerical_data_X.columns), axis=1)


In [141]:
X, y = df_X, df_y

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

In [176]:
import xlsxwriter
import contextlib
import os

def export_mean_sd(df_all, df_train, df_test, filename):


    with contextlib.suppress(FileNotFoundError):
        os.remove(filename)

    result_all = df_all.agg(['mean', 'std'])
    result_train = df_train.agg(['mean', 'std'])
    result_test = df_test.agg(['mean', 'std'])

    # create a new Excel workbook and add a worksheet
    workbook = xlsxwriter.Workbook(filename)
    worksheet1 = workbook.add_worksheet("Lab")
    worksheet2 = workbook.add_worksheet("Categorical")

    # write the mean and standard deviation to the worksheet
    worksheet1.write('A1', 'Variable')
    worksheet1.write('B1', 'Mean(SD)')
    worksheet1.write('C1', 'Mean(SD)')
    worksheet1.write('D1', 'Mean(SD)')

    worksheet2.write('A1', 'Variable')
    worksheet2.write('B1', 'N(%)')
    worksheet2.write('C1', 'N(%)')
    worksheet2.write('D1', 'N(%)')

    for i, col in enumerate(df_all.columns):
        worksheet1.write(i+1, 0, col)

        
        if col not in categorical_X:
            worksheet1.write(i+1, 0, col)    
            worksheet1.write(i+1, 1, str(round(result_all.loc['mean', col],2))+ '('+str(round(result_all.loc['std', col],2))+')')
            worksheet1.write(i+1, 2, str(round(result_train.loc['mean', col],2))+ '('+str(round(result_train.loc['std', col],2))+')')
            worksheet1.write(i+1, 3, str(round(result_test.loc['mean', col],2))+ '('+str(round(result_test.loc['std', col],2))+')')

        else:
            worksheet2.write(i+1, 0, col)

            value_counts_all = df_all[col].value_counts()
            # value_percentages_all = 100 * value_counts_all / len(df_all)

            value_counts_train = df_train[col].value_counts()
            # value_percentages_train = 100 * value_counts_train / len(df_train)

            value_counts_test = df_test[col].value_counts()
            # value_percentages_test = 100 * value_counts_test / len(df_test)

            col_all = ""
            col_train = ""
            col_test = ""

            for val, cnt in value_counts_all.iteritems():
                col_all = col_all + str(cnt) + "(" + str(round(100*(cnt/len(df_all)),2))+"%)" + "\n"
            
            for val, cnt in value_counts_train.iteritems():
                col_train = col_train + str(cnt) + "(" + str(round(100*(cnt/len(df_all)),2))+"%)" + "\n"
            
            for val, cnt in value_counts_test.iteritems():
                col_test = col_test + str(cnt) + "(" + str(round(100*(cnt/len(df_all)),2))+"%)" + "\n"
            
            worksheet2.write(i+1, 1, col_all)
            worksheet2.write(i+1, 2, col_train)
            worksheet2.write(i+1, 3, col_test)

    workbook.close()

In [177]:
export_mean_sd(df_X, X_train, X_test, 'All_data.xlsx')